In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score

In [30]:
cols = ['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription', 'spn', 'fmi', 'active', 'activeTransitionCount',
           'EquipmentID', 'Latitude', 'Longitude']

faults = pd.read_csv("../data/J1939Faults.csv", usecols = cols, dtype = str)
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851
1,2,990360,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.42125,-87.767361
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361
4,5,990416,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638


In [31]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   RecordID               1187335 non-null  object
 1   ESS_Id                 1187335 non-null  object
 2   EventTimeStamp         1187335 non-null  object
 3   eventDescription       1126490 non-null  object
 4   spn                    1187335 non-null  object
 5   fmi                    1187335 non-null  object
 6   active                 1187335 non-null  object
 7   activeTransitionCount  1187335 non-null  object
 8   EquipmentID            1187335 non-null  object
 9   Latitude               1187335 non-null  object
 10  Longitude              1187335 non-null  object
dtypes: object(11)
memory usage: 99.6+ MB


In [32]:
# convert dtypes as needed
# some columns have numeric-appearing values but they're not continuous variables, leaving them as strings
faults['EventTimeStamp'] = pd.to_datetime(faults['EventTimeStamp'])
faults['activeTransitionCount'] = faults['activeTransitionCount'].astype(int)

In [33]:
# time of day for faults could be interesting, split event column to date and timestamp columns
faults['event_date'] = faults['EventTimeStamp'].dt.date
faults['event_time'] = faults['EventTimeStamp'].dt.time

In [34]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,event_date,event_time
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,2015-02-21,10:47:13
1,2,990360,2015-02-21 11:34:34,NaN,629,12,True,127,1439,38.857638,-84.626851,2015-02-21,11:34:34
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.42125,-87.767361,2015-02-21,11:35:31
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21,11:35:33
4,5,990416,2015-02-21 11:39:41,NaN,4364,17,False,2,1674,38.416481,-89.442638,2015-02-21,11:39:41


In [35]:
# sort df by equipment, spn, timestamp
faults = faults.copy()
faults = faults.sort_values(by = ['EquipmentID', 'spn', 'EventTimeStamp'])

# get index for first row in each group, yields list of integers
first_index = faults.groupby(['EquipmentID', 'spn']).head(1).index

# check first_index for rows where active == False
drop_index = first_index[faults.loc[first_index, 'active'] == 'False']

# drop rows where first index is an active False row
faults = faults.drop(drop_index)

# now this should work as expected
faults['false_eventTimeStamp'] = faults.sort_values(by = ['EventTimeStamp']).groupby(by = ['EquipmentID', 'spn'])['EventTimeStamp'].shift(-1)

In [36]:
# test to reveal if the process worked (this group previously had a False as the first timestamp)
faults.loc[(faults['EquipmentID'] == '1369') & (faults['spn'] == '1807')].sort_values(by = ['EventTimeStamp'])

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,event_date,event_time,false_eventTimeStamp
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21,11:35:33,2015-02-21 11:57:37
31,32,990702,2015-02-21 11:57:37,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.42787,-87.756759,2015-02-21,11:57:37,2015-02-21 12:13:47
49,50,990999,2015-02-21 12:13:47,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.431574,-87.758981,2015-02-21,12:13:47,2015-02-21 18:26:34
421,422,995975,2015-02-21 18:26:34,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,38.330833,-85.757037,2015-02-21,18:26:34,2015-02-21 18:26:37
422,423,995979,2015-02-21 18:26:37,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,38.330833,-85.757083,2015-02-21,18:26:37,2015-02-21 18:32:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6426,6427,1088496,2015-02-26 12:14:25,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,36.936018,-86.50726800000001,2015-02-26,12:14:25,2015-02-26 13:00:12
6434,6435,1089226,2015-02-26 13:00:12,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,36.717453000000006,-86.525,2015-02-26,13:00:12,2015-02-26 13:11:23
6437,6438,1089547,2015-02-26 13:11:23,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,36.718148,-86.525324,2015-02-26,13:11:23,2015-02-26 16:05:38
6492,6493,1094483,2015-02-26 16:05:38,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,35.679212,-88.745046,2015-02-26,16:05:38,2015-02-26 16:05:41


In [ ]:
# lamp duration column (how much time was the light on for a fault)


In [37]:
# mask for each service station, lat and long to 2 decimal places
station_1 = (faults['Latitude'].str.contains('36.06')) & (faults['Longitude'].str.contains('86.43'))
station_2 = (faults['Latitude'].str.contains('35.58')) & (faults['Longitude'].str.contains('86.44'))
station_3 = (faults['Latitude'].str.contains('36.19')) & (faults['Longitude'].str.contains('83.17'))

# dataframe without faults associated with service locations
faults_nonservice = faults[~(station_1 | station_2 | station_3)]

### next steps
- add date and time breakout columns (year, month, date, weekday, hour, day segment (divide 24 hours into quadrants))
- merge diagnostic data with filtered faults data (inner join)
- figure out imputing methods
- for models, imput after train/test split
- do some eda on the data

In [38]:
faults_nonservice = faults_nonservice.copy()

faults_nonservice['event_year'] = faults_nonservice['EventTimeStamp'].dt.year
faults_nonservice['event_month'] = faults_nonservice['EventTimeStamp'].dt.month
faults_nonservice['event_day'] = faults_nonservice['EventTimeStamp'].dt.day
faults_nonservice['event_dayofweek'] = faults_nonservice['EventTimeStamp'].dt.dayofweek
faults_nonservice['event_dayname'] = faults_nonservice['EventTimeStamp'].dt.day_name()
faults_nonservice['event_hour'] = faults_nonservice['EventTimeStamp'].dt.hour
faults_nonservice['event_time_quadrant'] = faults_nonservice['EventTimeStamp'].dt.hour // 4

In [39]:
faults_nonservice.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,...,event_date,event_time,false_eventTimeStamp,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant
1001106,1038243,55748536,2018-07-20 09:31:33,High (Severity Medium) J1939 Network #2,1231,16,True,2,0105406655,36.139351,...,2018-07-20,09:31:33,2018-07-20 09:42:30,2018,7,20,4,Friday,9,2
1001107,1038244,55748640,2018-07-20 09:42:30,High (Severity Medium) J1939 Network #2,1231,16,False,2,0105406655,36.138981,...,2018-07-20,09:42:30,NaT,2018,7,20,4,Friday,9,2
358800,366301,7171498,2016-01-31 07:12:25,NaN,629,12,True,127,105301976,41.987175,...,2016-01-31,07:12:25,NaT,2016,1,31,6,Sunday,7,1
898852,923783,33800297,2017-12-04 22:25:09,Low (Severity Medium) Engine Coolant Level,111,18,False,1,105311240,35.609444,...,2017-12-04,22:25:09,NaT,2017,12,4,0,Monday,22,5
1093996,1141606,81618595,2019-03-27 08:10:52,Low (Severity Medium) Engine Coolant Level,111,18,True,1,105338729,36.060324,...,2019-03-27,08:10:52,2019-03-27 08:12:58,2019,3,27,2,Wednesday,8,2


In [62]:
# drop active == False (Kagon is keeping these rows so it will give us a good way to compare performance)
faults_active = faults_nonservice.loc[faults_nonservice['active'] == 'True']

In [73]:
# for each equipment id that experienced a derate, durration of time before derate for each fault
# complicated by equipment that has more than one derate event
# derates can occur at any time of day (tend to be clustered during working hours bet that's probably because there are more trucks active) so don't want to just group by day
# just to get something working, go on date for now
faults_active = faults_active.copy()
faults_active['derate_true'] = np.where(faults_active.groupby(['EquipmentID', 'event_date'])['spn'].transform(lambda x: (x == '5246').any()), 'True', 'False')
# once we get how to calculate time of event before derate this code can be modified slightly to detect

In [87]:
# partial derate occurring on the same day as a full derate - incorporate this as a signal boost?
# come back to this after initial model
faults_active.loc[(faults_active['spn'] == '1569') & (faults_active['derate_true'] == 'True')]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,...,event_time,false_eventTimeStamp,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,derate_true
996749,1032907,54267109,2018-07-06 04:53:05,Condition Exists Engine Protection Torque Derate,1569,31,True,2,105349576,36.066712,...,04:53:05,2018-07-10 15:44:13,2018,7,6,4,Friday,4,1,True
5714,5715,1070647,2015-02-25 13:53:08,Condition Exists Engine Protection Torque Derate,1569,31,True,1,1329,39.399583,...,13:53:08,2015-02-25 14:47:20,2015,2,25,2,Wednesday,13,3,True
82873,85259,2919536,2015-06-12 08:24:15,Condition Exists Engine Protection Torque Derate,1569,31,True,1,1339,36.822962,...,08:24:15,2015-06-14 15:56:28,2015,6,12,4,Friday,8,2,True
82607,84993,2915364,2015-06-12 03:57:49,Condition Exists Engine Protection Torque Derate,1569,31,True,1,1366,35.194398,...,03:57:49,2015-06-12 07:18:46,2015,6,12,4,Friday,3,0,True
83619,86005,2932223,2015-06-12 19:26:10,Condition Exists Engine Protection Torque Derate,1569,31,True,1,1366,35.175,...,19:26:10,2015-06-12 20:49:02,2015,6,12,4,Friday,19,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125718,1177138,93351982,2019-07-09 00:11:00,Condition Exists Engine Protection Torque Derate,1569,31,True,1,1991,30.989537,...,00:11:00,2019-07-11 13:30:47,2019,7,9,1,Tuesday,0,0,True
964660,995168,45003043,2018-04-07 02:07:44,Condition Exists Engine Protection Torque Derate,1569,31,True,2,2007,37.816435,...,02:07:44,2018-04-07 09:45:39,2018,4,7,5,Saturday,2,0,True
794693,815232,18884125,2017-06-24 10:42:36,Condition Exists Engine Protection Torque Derate,1569,31,True,1,2009,33.416296,...,10:42:36,2017-06-24 15:14:43,2017,6,24,5,Saturday,10,2,True
1139261,1192545,100080700,2019-08-30 18:05:29,Condition Exists Engine Protection Torque Derate,1569,31,True,1,2021,39.62,...,18:05:29,2019-09-06 23:47:17,2019,8,30,4,Friday,18,4,True


In [41]:
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [74]:
diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [77]:
# pivot on distinct values in 'Name'
diagnostics['Name'].unique()

array(['IgnStatus', 'EngineOilPressure', 'EngineOilTemperature',
       'TurboBoostPressure', 'EngineLoad', 'AcceleratorPedal',
       'IntakeManifoldTemperature', 'FuelRate', 'FuelLtd', 'EngineRpm',
       'LampStatus', 'BarometricPressure', 'FuelLevel', 'Speed',
       'EngineTimeLtd', 'CruiseControlSetSpeed', 'CruiseControlActive',
       'EngineCoolantTemperature', 'ParkingBrake',
       'SwitchedBatteryVoltage', 'DistanceLtd', 'Throttle',
       'FuelTemperature', 'ServiceDistance'], dtype=object)

In [92]:
diag_pivot = diagnostics.pivot(index = ['FaultId'], columns = ['Name'], values = ['Value'])
diag_pivot.columns = diag_pivot.columns.droplevel()

In [93]:
diag_pivot = diag_pivot.reset_index().rename_axis(None, axis = 1)


In [97]:
diag_pivot['FaultId'] = diag_pivot['FaultId'].astype(str)

In [102]:
# merge faults_active to diag_pivot on RecordID = FaultId
# how = left (keeps active True from faults and drops additional rows from diagnostics)
fault_diag = pd.merge(faults_active, diag_pivot, how = 'left', left_on = 'RecordID', right_on = 'FaultId')

In [108]:
fault_diag.isnull().sum()
# total rows ~550K
# rows that definitely don't have enough values to be useful: ServiceDistance, SwitchedBatteryVoltage
# extremely unlikely to be useful (1/3 to over half missing): FuelTemperature, ParkingBrake, Throttle
# iffy but possible to imput with some degree of meaning: FuelLevel, AcceleratorPedal
# quite a few are around 20-30K missing values

RecordID                          0
ESS_Id                            0
EventTimeStamp                    0
eventDescription              29996
spn                               0
fmi                               0
active                            0
activeTransitionCount             0
EquipmentID                       0
Latitude                          0
Longitude                         0
event_date                        0
event_time                        0
false_eventTimeStamp            934
event_year                        0
event_month                       0
event_day                         0
event_dayofweek                   0
event_dayname                     0
event_hour                        0
event_time_quadrant               0
derate_true                       0
FaultId                           0
AcceleratorPedal              68105
BarometricPressure            17752
CruiseControlActive           27913
CruiseControlSetSpeed         26855
DistanceLtd                 

In [111]:
# drop columns that are unlikely to be meaningful
fault_diag = fault_diag.drop(['ServiceDistance', 'SwitchedBatteryVoltage', 'FuelTemperature', 'ParkingBrake', 'Throttle', 'FaultId'], axis = 1)

In [115]:
fault_diag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549709 entries, 0 to 549708
Data columns (total 41 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   RecordID                   549709 non-null  object        
 1   ESS_Id                     549709 non-null  object        
 2   EventTimeStamp             549709 non-null  datetime64[ns]
 3   eventDescription           519713 non-null  object        
 4   spn                        549709 non-null  object        
 5   fmi                        549709 non-null  object        
 6   active                     549709 non-null  object        
 7   activeTransitionCount      549709 non-null  int64         
 8   EquipmentID                549709 non-null  object        
 9   Latitude                   549709 non-null  object        
 10  Longitude                  549709 non-null  object        
 11  event_date                 549709 non-null  object  

In [118]:
# decide on X (variables) and y (target)
X = ['spn', 'activeTransitionCount', 'EquipmentID', 'event_time_quadrant'] # minimal for first go, no missing values
categorical_variables = ['spn', 'EquipmentID', 'event_time_quadrant']
y = fault_diag['derate_true']

# split data into train/test before scaling and imputation
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False, drop = 'first'), categorical_variables)
            ],
            remainder = 'passthrough', verbose_feature_names_out = False)),
        ('pf', PolynomialFeatures(interaction_only = True, include_bias = False)),
        ('vt', VarianceThreshold()),
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression())
    ]
)

pipe.fit(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [4, 549709]

### target:
- group by equipment ID and sort by event datetime
- if equipment had derate (fault id) boolean column true for records within a certain timeframe prior
- looking for a signal with enough time to get the truck to a service location

### predictors:
- fault codes, diagnostics (downside is that a lot of values are missing, but they seem to be more present with active true)
- might not be value in looking at active false (Tomo's group only looked at active true)

### do this as part of data cleaning:
- pull timestamp from active false to get duration of active true (there should be paired rows based on fault code, different lights for different faults)
- do this before dropping service locations (don't want to miss active false which happened as a result of service)

### other thoughts:
- Code P0606 tends to be set when a PCM/ECM has failed. Depending on component condition and the make and model of a vehicle, it may be possible to resolve an internal integrity fault in the PCM/ECM by upgrading or re-flashing the control module.
- since it's a fault with the monitoring device itself, might not have any predictive power
- how many are there?
- if we get rid of this (and other faults not actually related to the engine) might reduce noise in the dataset?
- since there are active true with no diagnostics, could the rows with null diagnostics be faults like this one that don't have anything to do with the engine?